# 2_Merge_Months_By_Join.ipynb

Take in the cleaned, processed Valuations files month by month.

 For each month:
  - The data is already sorted by Policy_Number and Fund_Code.
  - Summarise by policy number and concatenate all the fund codes for that policy for that month onto a single row, into a new column, given a name the same as the YYYY_MM Value of the data.
  - Now merge the files horizontally, month on month to build up a timeline.
 
Then we end up with a single dataframe which has the files for each system merged with the same structure and then concatenated into a single file representing a 15 month timeline of fund investment.  

This is written to a csv file for later processing.

In [1]:
# importing the librarIES
import os                           # For file processing.
import csv
import pandas as pd            # For dataframes.
from pathlib import Path
import pyodbc                       # For connecting to SQL Server
import datetime
import numpy
import psutil 
import gc
import hashlib

# importing matplotlib module
import matplotlib.pyplot as plt
plt.style.use('default')
  
# %matplotlib inline: only draw static
# images in the notebook
%matplotlib inline

## Set all runtime variables.

In [2]:
e = datetime.datetime.now()
print ("Current date and time = %s" % e)
print ("Current memory available = ",psutil.virtual_memory().available * 100 / psutil.virtual_memory().total)

# File names
# Set values for the various paths.
merged_path = ".\data\merged"

# Set values for the various file extension types.
out_file_type = ".csv"

# Set values for the various file prefixes.
merged_file_prefix = "Fund_Investment_History_"

Current date and time = 2023-04-22 12:39:21.168204
Current memory available =  26.65724599633733


# 1. Read in the first month, 2022-01

In [3]:
# Read the data into a pandas dataframe
yearmonth = "202201"
Aggregated_Merged_Filename = os.path.join(merged_path, (merged_file_prefix + yearmonth + out_file_type))
print("Aggregated_Merged_Filename-",Aggregated_Merged_Filename)

df_in = pd.read_csv(Aggregated_Merged_Filename, delimiter=',',low_memory=False, header=0)
df_in.head(10)

Aggregated_Merged_Filename- .\data\merged\Fund_Investment_History_202201.csv


,POL_NUMB,API_TRANCHE_EXT,DUB_NAME,RUN_MONTH
0,001045160#,132935.5,TSDGBTSDITTSEETTSSITTSYGB,202201
1,001045178#,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,202201
2,001046154#,7669.3,TSESS,202201
3,001047062#,2556.4,TSDGB,202201
4,001049026#,2556.4,TSESD,202201
5,001050063#,7669.2,TSEAMTSEETTSESS,202201
6,001050203#,20656.0,TSDGBTSDITTSESSTSYGB,202201
7,001052119#,7669.2,TSDGBTSEETTSESS,202201
8,001056186#,2556.4,TUASC,202201
9,001056204#,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,202201


In [4]:
df_in.dtypes

POL_NUMB            object
API_TRANCHE_EXT    float64
DUB_NAME            object
RUN_MONTH            int64
dtype: object

# 2. Concatenate the next month, 2022-02

In [5]:
# Read the data into a pandas dataframe
yearmonth = "202202"
Aggregated_Merged_Filename = os.path.join(merged_path, (merged_file_prefix + yearmonth + out_file_type))
print("Aggregated_Merged_Filename-",Aggregated_Merged_Filename)
df_in1 = pd.read_csv(Aggregated_Merged_Filename, delimiter=',',low_memory=False, header=0)
# Merge the first and second years
df_merged = pd.merge(df_in, df_in1, on=['POL_NUMB'], how='outer')
df_merged.rename(columns = {'DUB_NAME_x':'202201','DUB_NAME_y':yearmonth}, inplace = True)
df_merged = df_merged.drop(columns=['RUN_MONTH_x', 'RUN_MONTH_y'])
df_merged.head(10)

Aggregated_Merged_Filename- .\data\merged\Fund_Investment_History_202202.csv


,POL_NUMB,API_TRANCHE_EXT_x,202201,API_TRANCHE_EXT_y,202202
0,001045160#,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB
1,001045178#,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB
2,001046154#,7669.3,TSESS,7669.3,TSESS
3,001047062#,2556.4,TSDGB,2556.4,TSDGB
4,001049026#,2556.4,TSESD,2556.4,TSESD
5,001050063#,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS
6,001050203#,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB
7,001052119#,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS
8,001056186#,2556.4,TUASC,2556.4,TUASC
9,001056204#,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB


# 3. Now do the rest.

In [6]:
# Read in and join 202203
yearmonth = "202203"
Aggregated_Merged_Filename = os.path.join(merged_path, (merged_file_prefix + yearmonth + out_file_type))
print("Aggregated_Merged_Filename-",Aggregated_Merged_Filename)
df_in2 = pd.read_csv(Aggregated_Merged_Filename, delimiter=',',low_memory=False, header=0)

# Merge the years
df_merged2 = pd.merge(df_merged, df_in2, on=['POL_NUMB'], how='outer')
df_merged2.rename(columns = {'DUB_NAME':yearmonth}, inplace = True)
df_merged2 = df_merged2.drop(columns=['RUN_MONTH'])
df_merged2.head(10)

Aggregated_Merged_Filename- .\data\merged\Fund_Investment_History_202203.csv


,POL_NUMB,API_TRANCHE_EXT_x,202201,API_TRANCHE_EXT_y,202202,API_TRANCHE_EXT,202203
0,001045160#,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB
1,001045178#,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB
2,001046154#,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS
3,001047062#,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB
4,001049026#,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD
5,001050063#,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS
6,001050203#,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB
7,001052119#,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS
8,001056186#,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC
9,001056204#,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB


### Read in and join 202204

In [7]:
yearmonth = "202204"
Aggregated_Merged_Filename = os.path.join(merged_path, (merged_file_prefix + yearmonth + out_file_type))
print("Aggregated_Merged_Filename-",Aggregated_Merged_Filename)
df_in3 = pd.read_csv(Aggregated_Merged_Filename, delimiter=',',low_memory=False, header=0)
# Merge the years
df_merged3 = pd.merge(df_merged2, df_in3, on=['POL_NUMB'], how='outer')
df_merged3.rename(columns = {'DUB_NAME':yearmonth}, inplace = True)
df_merged3 = df_merged3.drop(columns=['RUN_MONTH'])
df_merged3.head(10)

Aggregated_Merged_Filename- .\data\merged\Fund_Investment_History_202204.csv


,POL_NUMB,API_TRANCHE_EXT_x,202201,API_TRANCHE_EXT_y,202202,API_TRANCHE_EXT_x,202203,API_TRANCHE_EXT_y,202204
0,001045160#,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB
1,001045178#,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB
2,001046154#,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS
3,001047062#,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB
4,001049026#,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD
5,001050063#,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS
6,001050203#,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB
7,001052119#,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS
8,001056186#,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC
9,001056204#,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB


### Read in and join 202205

In [8]:
yearmonth = "202205"
Aggregated_Merged_Filename = os.path.join(merged_path, (merged_file_prefix + yearmonth + out_file_type))
print("Aggregated_Merged_Filename-",Aggregated_Merged_Filename)
df_in4 = pd.read_csv(Aggregated_Merged_Filename, delimiter=',',low_memory=False, header=0)
# Merge the years
df_merged4 = pd.merge(df_merged3, df_in4, on=['POL_NUMB'], how='outer')
df_merged4.rename(columns = {'DUB_NAME':yearmonth}, inplace = True)
df_merged4 = df_merged4.drop(columns=['RUN_MONTH'])
df_merged4.head(10)

Aggregated_Merged_Filename- .\data\merged\Fund_Investment_History_202205.csv


,POL_NUMB,API_TRANCHE_EXT_x,202201,API_TRANCHE_EXT_y,202202,API_TRANCHE_EXT_x,202203,API_TRANCHE_EXT_y,202204,API_TRANCHE_EXT,202205
0,001045160#,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB
1,001045178#,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB
2,001046154#,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS
3,001047062#,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB
4,001049026#,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD
5,001050063#,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS
6,001050203#,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB
7,001052119#,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS
8,001056186#,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC
9,001056204#,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB


### Read in and join 202206

In [9]:
yearmonth = "202206"
Aggregated_Merged_Filename = os.path.join(merged_path, (merged_file_prefix + yearmonth + out_file_type))
print("Aggregated_Merged_Filename-",Aggregated_Merged_Filename)
df_in5 = pd.read_csv(Aggregated_Merged_Filename, delimiter=',',low_memory=False, header=0)
# Merge the years
df_merged5 = pd.merge(df_merged4, df_in5, on=['POL_NUMB'], how='outer')
df_merged5.rename(columns = {'DUB_NAME':yearmonth}, inplace = True)
df_merged5 = df_merged5.drop(columns=['RUN_MONTH'])
df_merged5.head(10)

Aggregated_Merged_Filename- .\data\merged\Fund_Investment_History_202206.csv


,POL_NUMB,API_TRANCHE_EXT_x,202201,API_TRANCHE_EXT_y,202202,API_TRANCHE_EXT_x,202203,API_TRANCHE_EXT_y,202204,API_TRANCHE_EXT_x,202205,API_TRANCHE_EXT_y,202206
0,001045160#,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB
1,001045178#,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB
2,001046154#,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS
3,001047062#,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB
4,001049026#,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD
5,001050063#,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS
6,001050203#,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB
7,001052119#,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS
8,001056186#,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC
9,001056204#,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB


### Read in and join 202207

In [10]:
yearmonth = "202207"
Aggregated_Merged_Filename = os.path.join(merged_path, (merged_file_prefix + yearmonth + out_file_type))
print("Aggregated_Merged_Filename-",Aggregated_Merged_Filename)
df_in6 = pd.read_csv(Aggregated_Merged_Filename, delimiter=',',low_memory=False, header=0)
# Merge the years
df_merged6 = pd.merge(df_merged5, df_in6, on=['POL_NUMB'], how='outer')
df_merged6.rename(columns = {'DUB_NAME':yearmonth}, inplace = True)
df_merged6 = df_merged6.drop(columns=['RUN_MONTH'])
df_merged6.head(10)

Aggregated_Merged_Filename- .\data\merged\Fund_Investment_History_202207.csv


,POL_NUMB,API_TRANCHE_EXT_x,202201,API_TRANCHE_EXT_y,202202,API_TRANCHE_EXT_x,202203,API_TRANCHE_EXT_y,202204,API_TRANCHE_EXT_x,202205,API_TRANCHE_EXT_y,202206,API_TRANCHE_EXT,202207
0,001045160#,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB
1,001045178#,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB
2,001046154#,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS
3,001047062#,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB
4,001049026#,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD
5,001050063#,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS
6,001050203#,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB
7,001052119#,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS
8,001056186#,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC
9,001056204#,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB


### Read in and join 202208

In [11]:
yearmonth = "202208"
Aggregated_Merged_Filename = os.path.join(merged_path, (merged_file_prefix + yearmonth + out_file_type))
print("Aggregated_Merged_Filename-",Aggregated_Merged_Filename)
df_in7 = pd.read_csv(Aggregated_Merged_Filename, delimiter=',',low_memory=False, header=0)
# Merge the years
df_merged7 = pd.merge(df_merged6, df_in7, on=['POL_NUMB'], how='outer')
df_merged7.rename(columns = {'DUB_NAME':yearmonth}, inplace = True)
df_merged7 = df_merged7.drop(columns=['RUN_MONTH'])
df_merged7.head(10)

Aggregated_Merged_Filename- .\data\merged\Fund_Investment_History_202208.csv


,POL_NUMB,API_TRANCHE_EXT_x,202201,API_TRANCHE_EXT_y,202202,API_TRANCHE_EXT_x,202203,API_TRANCHE_EXT_y,202204,API_TRANCHE_EXT_x,202205,API_TRANCHE_EXT_y,202206,API_TRANCHE_EXT_x,202207,API_TRANCHE_EXT_y,202208
0,001045160#,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB
1,001045178#,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB
2,001046154#,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS
3,001047062#,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB
4,001049026#,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD
5,001050063#,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS
6,001050203#,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB
7,001052119#,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS
8,001056186#,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC,NaN,NaN
9,001056204#,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB


### Read in and join 202209

In [12]:
yearmonth = "202209"
Aggregated_Merged_Filename = os.path.join(merged_path, (merged_file_prefix + yearmonth + out_file_type))
print("Aggregated_Merged_Filename-",Aggregated_Merged_Filename)
df_in8 = pd.read_csv(Aggregated_Merged_Filename, delimiter=',',low_memory=False, header=0)
# Merge the years
df_merged8 = pd.merge(df_merged7, df_in8, on=['POL_NUMB'], how='outer')
df_merged8.rename(columns = {'DUB_NAME':yearmonth}, inplace = True)
df_merged8 = df_merged8.drop(columns=['RUN_MONTH'])
df_merged8.head(10)

Aggregated_Merged_Filename- .\data\merged\Fund_Investment_History_202209.csv


,POL_NUMB,API_TRANCHE_EXT_x,202201,API_TRANCHE_EXT_y,202202,API_TRANCHE_EXT_x,202203,API_TRANCHE_EXT_y,202204,API_TRANCHE_EXT_x,202205,API_TRANCHE_EXT_y,202206,API_TRANCHE_EXT_x,202207,API_TRANCHE_EXT_y,202208,API_TRANCHE_EXT,202209
0,001045160#,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB
1,001045178#,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB
2,001046154#,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS
3,001047062#,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB
4,001049026#,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD
5,001050063#,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS
6,001050203#,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB
7,001052119#,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS
8,001056186#,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC,NaN,NaN,NaN,NaN
9,001056204#,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB


### Read in and join 202210

In [13]:
yearmonth = "202210"
Aggregated_Merged_Filename = os.path.join(merged_path, (merged_file_prefix + yearmonth + out_file_type))
print("Aggregated_Merged_Filename-",Aggregated_Merged_Filename)
df_in9 = pd.read_csv(Aggregated_Merged_Filename, delimiter=',',low_memory=False, header=0)
# Merge the years
df_merged9 = pd.merge(df_merged8, df_in9, on=['POL_NUMB'], how='outer')
df_merged9.rename(columns = {'DUB_NAME':yearmonth}, inplace = True)
df_merged9 = df_merged9.drop(columns=['RUN_MONTH'])
df_merged9.head(10)

Aggregated_Merged_Filename- .\data\merged\Fund_Investment_History_202210.csv


,POL_NUMB,API_TRANCHE_EXT_x,202201,API_TRANCHE_EXT_y,202202,API_TRANCHE_EXT_x,202203,API_TRANCHE_EXT_y,202204,API_TRANCHE_EXT_x,...,API_TRANCHE_EXT_y,202206,API_TRANCHE_EXT_x,202207,API_TRANCHE_EXT_y,202208,API_TRANCHE_EXT_x,202209,API_TRANCHE_EXT_y,202210
0,001045160#,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,...,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB
1,001045178#,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,...,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB
2,001046154#,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,...,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS
3,001047062#,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,...,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB
4,001049026#,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,...,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD
5,001050063#,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,...,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS
6,001050203#,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,...,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB
7,001052119#,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,...,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS
8,001056186#,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC,2556.4,...,2556.4,TUASC,2556.4,TUASC,NaN,NaN,NaN,NaN,NaN,NaN
9,001056204#,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,...,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB


### Read in and join 202211

In [14]:
yearmonth = "202211"
Aggregated_Merged_Filename = os.path.join(merged_path, (merged_file_prefix + yearmonth + out_file_type))
print("Aggregated_Merged_Filename-",Aggregated_Merged_Filename)
df_in10 = pd.read_csv(Aggregated_Merged_Filename, delimiter=',',low_memory=False, header=0)
# Merge the years
df_merged10 = pd.merge(df_merged9, df_in10, on=['POL_NUMB'], how='outer')
df_merged10.rename(columns = {'DUB_NAME':yearmonth}, inplace = True)
df_merged10 = df_merged10.drop(columns=['RUN_MONTH'])
df_merged10.head(10)

Aggregated_Merged_Filename- .\data\merged\Fund_Investment_History_202211.csv


,POL_NUMB,API_TRANCHE_EXT_x,202201,API_TRANCHE_EXT_y,202202,API_TRANCHE_EXT_x,202203,API_TRANCHE_EXT_y,202204,API_TRANCHE_EXT_x,...,API_TRANCHE_EXT_x,202207,API_TRANCHE_EXT_y,202208,API_TRANCHE_EXT_x,202209,API_TRANCHE_EXT_y,202210,API_TRANCHE_EXT,202211
0,001045160#,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,...,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB
1,001045178#,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,...,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB
2,001046154#,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,...,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS
3,001047062#,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,...,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB
4,001049026#,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,...,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD
5,001050063#,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,...,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS
6,001050203#,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,...,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB
7,001052119#,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,...,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS
8,001056186#,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC,2556.4,...,2556.4,TUASC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,001056204#,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,...,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB


### Read in and join 202212

In [15]:
yearmonth = "202212"
Aggregated_Merged_Filename = os.path.join(merged_path, (merged_file_prefix + yearmonth + out_file_type))
print("Aggregated_Merged_Filename-",Aggregated_Merged_Filename)
df_in11 = pd.read_csv(Aggregated_Merged_Filename, delimiter=',',low_memory=False, header=0)
# Merge the years
df_merged11 = pd.merge(df_merged10, df_in11, on=['POL_NUMB'], how='outer')
df_merged11.rename(columns = {'DUB_NAME':yearmonth}, inplace = True)
df_merged11 = df_merged11.drop(columns=['RUN_MONTH'])
df_merged11.head(10)

Aggregated_Merged_Filename- .\data\merged\Fund_Investment_History_202212.csv


,POL_NUMB,API_TRANCHE_EXT_x,202201,API_TRANCHE_EXT_y,202202,API_TRANCHE_EXT_x,202203,API_TRANCHE_EXT_y,202204,API_TRANCHE_EXT_x,...,API_TRANCHE_EXT_y,202208,API_TRANCHE_EXT_x,202209,API_TRANCHE_EXT_y,202210,API_TRANCHE_EXT_x,202211,API_TRANCHE_EXT_y,202212
0,001045160#,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,...,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB
1,001045178#,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,...,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB
2,001046154#,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,...,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS
3,001047062#,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,...,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB
4,001049026#,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,...,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD
5,001050063#,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,...,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS
6,001050203#,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,...,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB
7,001052119#,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,...,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS
8,001056186#,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC,2556.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,001056204#,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,...,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB


### Read in and join 202301

In [16]:
yearmonth = "202301"
Aggregated_Merged_Filename = os.path.join(merged_path, (merged_file_prefix + yearmonth + out_file_type))
print("Aggregated_Merged_Filename-",Aggregated_Merged_Filename)
df_in12 = pd.read_csv(Aggregated_Merged_Filename, delimiter=',',low_memory=False, header=0)
# Merge the years
df_merged12 = pd.merge(df_merged11, df_in12, on=['POL_NUMB'], how='outer')
df_merged12.rename(columns = {'DUB_NAME':yearmonth}, inplace = True)
df_merged12 = df_merged12.drop(columns=['RUN_MONTH'])
df_merged12.head(10)

Aggregated_Merged_Filename- .\data\merged\Fund_Investment_History_202301.csv


,POL_NUMB,API_TRANCHE_EXT_x,202201,API_TRANCHE_EXT_y,202202,API_TRANCHE_EXT_x,202203,API_TRANCHE_EXT_y,202204,API_TRANCHE_EXT_x,...,API_TRANCHE_EXT_x,202209,API_TRANCHE_EXT_y,202210,API_TRANCHE_EXT_x,202211,API_TRANCHE_EXT_y,202212,API_TRANCHE_EXT,202301
0,001045160#,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,...,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB
1,001045178#,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,...,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB
2,001046154#,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,...,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS
3,001047062#,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,...,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB
4,001049026#,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,...,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD
5,001050063#,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,...,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS
6,001050203#,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,...,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB
7,001052119#,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,...,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS
8,001056186#,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC,2556.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,001056204#,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,...,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB


### Read in and join 202302

In [23]:
yearmonth = "202302"
Aggregated_Merged_Filename = os.path.join(merged_path, (merged_file_prefix + yearmonth + out_file_type))
print("Aggregated_Merged_Filename-",Aggregated_Merged_Filename)
df_in13 = pd.read_csv(Aggregated_Merged_Filename, delimiter=',',low_memory=False, header=0)
# Merge the years
df_merged13 = pd.merge(df_merged12, df_in13, on=['POL_NUMB'], how='outer')
df_merged13.rename(columns = {'DUB_NAME':yearmonth}, inplace = True)
df_merged13 = df_merged13.drop(columns=['RUN_MONTH'])
df_merged13.head(10)

Aggregated_Merged_Filename- .\data\merged\Fund_Investment_History_202302.csv


,POL_NUMB,API_TRANCHE_EXT_x,202201,API_TRANCHE_EXT_y,202202,API_TRANCHE_EXT_x,202203,API_TRANCHE_EXT_y,202204,API_TRANCHE_EXT_x,...,API_TRANCHE_EXT_y,202210,API_TRANCHE_EXT_x,202211,API_TRANCHE_EXT_y,202212,API_TRANCHE_EXT_x,202301,API_TRANCHE_EXT_y,202302
0,001045160#,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,...,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB
1,001045178#,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,...,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB
2,001046154#,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,...,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS
3,001047062#,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,...,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB
4,001049026#,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,...,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD
5,001050063#,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,...,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS
6,001050203#,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,...,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB
7,001052119#,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,...,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS
8,001056186#,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC,2556.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,001056204#,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,...,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB


### Read in and join 202303

In [24]:
yearmonth = "202303"
Aggregated_Merged_Filename = os.path.join(merged_path, (merged_file_prefix + yearmonth + out_file_type))
print("Aggregated_Merged_Filename-",Aggregated_Merged_Filename)
df_in14 = pd.read_csv(Aggregated_Merged_Filename, delimiter=',',low_memory=False, header=0)
# Merge the years
df_merged14 = pd.merge(df_merged13, df_in14, on=['POL_NUMB'], how='outer')
df_merged14.rename(columns = {'DUB_NAME':yearmonth}, inplace = True)
df_merged14 = df_merged14.drop(columns=['RUN_MONTH'])
df_merged14.head(10)

Aggregated_Merged_Filename- .\data\merged\Fund_Investment_History_202303.csv


,POL_NUMB,API_TRANCHE_EXT_x,202201,API_TRANCHE_EXT_y,202202,API_TRANCHE_EXT_x,202203,API_TRANCHE_EXT_y,202204,API_TRANCHE_EXT_x,...,API_TRANCHE_EXT_x,202211,API_TRANCHE_EXT_y,202212,API_TRANCHE_EXT_x,202301,API_TRANCHE_EXT_y,202302,API_TRANCHE_EXT,202303
0,001045160#,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,...,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB
1,001045178#,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,...,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB
2,001046154#,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,...,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS,7669.3,TSESS
3,001047062#,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,...,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB,2556.4,TSDGB
4,001049026#,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,...,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD,2556.4,TSESD
5,001050063#,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,...,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS
6,001050203#,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,...,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB
7,001052119#,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,...,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS
8,001056186#,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC,2556.4,TUASC,2556.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,001056204#,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,...,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB


# 4. Remove all obsolete columns.

In [25]:
# Remove obsolete columns
df_merged14 = df_merged14.drop(columns=['API_TRANCHE_EXT_x', 'API_TRANCHE_EXT_y'])
df_merged14.head(10)

,POL_NUMB,202201,202202,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212,202301,202302,API_TRANCHE_EXT,202303
0,001045160#,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,132935.5,TSDGBTSDITTSEETTSSITTSYGB
1,001045178#,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.8,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB
2,001046154#,TSESS,TSESS,TSESS,TSESS,TSESS,TSESS,TSESS,TSESS,TSESS,TSESS,TSESS,TSESS,TSESS,TSESS,7669.3,TSESS
3,001047062#,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,2556.4,TSDGB
4,001049026#,TSESD,TSESD,TSESD,TSESD,TSESD,TSESD,TSESD,TSESD,TSESD,TSESD,TSESD,TSESD,TSESD,TSESD,2556.4,TSESD
5,001050063#,TSEAMTSEETTSESS,TSEAMTSEETTSESS,TSEAMTSEETTSESS,TSEAMTSEETTSESS,TSEAMTSEETTSESS,TSEAMTSEETTSESS,TSEAMTSEETTSESS,TSEAMTSEETTSESS,TSEAMTSEETTSESS,TSEAMTSEETTSESS,TSEAMTSEETTSESS,TSEAMTSEETTSESS,TSEAMTSEETTSESS,TSEAMTSEETTSESS,7669.2,TSEAMTSEETTSESS
6,001050203#,TSDGBTSDITTSESSTSYGB,TSDGBTSDITTSESSTSYGB,TSDGBTSDITTSESSTSYGB,TSDGBTSDITTSESSTSYGB,TSDGBTSDITTSESSTSYGB,TSDGBTSDITTSESSTSYGB,TSDGBTSDITTSESSTSYGB,TSDGBTSDITTSESSTSYGB,TSDGBTSDITTSESSTSYGB,TSDGBTSDITTSESSTSYGB,TSDGBTSDITTSESSTSYGB,TSDGBTSDITTSESSTSYGB,TSDGBTSDITTSESSTSYGB,TSDGBTSDITTSESSTSYGB,20656.0,TSDGBTSDITTSESSTSYGB
7,001052119#,TSDGBTSEETTSESS,TSDGBTSEETTSESS,TSDGBTSEETTSESS,TSDGBTSEETTSESS,TSDGBTSEETTSESS,TSDGBTSEETTSESS,TSDGBTSEETTSESS,TSDGBTSEETTSESS,TSDGBTSEETTSESS,TSDGBTSEETTSESS,TSDGBTSEETTSESS,TSDGBTSEETTSESS,TSDGBTSEETTSESS,TSDGBTSEETTSESS,7669.2,TSDGBTSEETTSESS
8,001056186#,TUASC,TUASC,TUASC,TUASC,TUASC,TUASC,TUASC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,001056204#,TSDGBTSEDETSEETTSESDTSESSTSYGB,TSDGBTSEDETSEETTSESDTSESSTSYGB,TSDGBTSEDETSEETTSESDTSESSTSYGB,TSDGBTSEDETSEETTSESDTSESSTSYGB,TSDGBTSEDETSEETTSESDTSESSTSYGB,TSDGBTSEDETSEETTSESDTSESSTSYGB,TSDGBTSEDETSEETTSESDTSESSTSYGB,TSDGBTSEDETSEETTSESDTSESSTSYGB,TSDGBTSEDETSEETTSESDTSESSTSYGB,TSDGBTSEDETSEETTSESDTSESSTSYGB,TSDGBTSEDETSEETTSESDTSESSTSYGB,TSDGBTSEDETSEETTSESDTSESSTSYGB,TSDGBTSEDETSEETTSESDTSESSTSYGB,TSDGBTSEDETSEETTSESDTSESSTSYGB,15338.4,TSDGBTSEDETSEETTSESDTSESSTSYGB


# 5. Write out the enhanced data for future use.

In [26]:
# Fill numeric NaNs with a zero
df_merged14['API_TRANCHE_EXT'] = df_merged14['API_TRANCHE_EXT'].fillna(0)

# Fill all the NaNs with an X
df_merged14.fillna(value = 'x',inplace = True)
df_merged14

,POL_NUMB,202201,202202,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212,202301,202302,API_TRANCHE_EXT,202303
0,001045160#,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,TSDGBTSDITTSEETTSSITTSYGB,132935.50,TSDGBTSDITTSEETTSSITTSYGB
1,001045178#,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB,17894.80,TSDGBTSEAMTSEDETSEETTSESSTSSGBTSYGB
2,001046154#,TSESS,TSESS,TSESS,TSESS,TSESS,TSESS,TSESS,TSESS,TSESS,TSESS,TSESS,TSESS,TSESS,TSESS,7669.30,TSESS
3,001047062#,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,TSDGB,2556.40,TSDGB
4,001049026#,TSESD,TSESD,TSESD,TSESD,TSESD,TSESD,TSESD,TSESD,TSESD,TSESD,TSESD,TSESD,TSESD,TSESD,2556.40,TSESD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265863,8157682K,x,x,x,x,x,x,x,x,x,x,x,x,x,x,6768.00,XBEAAXBEAS
265864,8157702V,x,x,x,x,x,x,x,x,x,x,x,x,x,x,1200.00,5024A
265865,8157705J,x,x,x,x,x,x,x,x,x,x,x,x,x,x,2400.00,XBEAAXBEAS
265866,8157728W,x,x,x,x,x,x,x,x,x,x,x,x,x,x,2802.96,5024A


In [27]:
# File names
Full_History_Filename = 'C:/Users/m033\MSc\Big_Data_Project\data\output/Full_Fund_Investment_History.csv'

# Write to a csv file and use index=false to drop teh index from the dataframe
df_merged14.to_csv(Full_History_Filename, encoding='utf-8', index=False)